In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score
import time
from torch.utils.data import Dataset, DataLoader


In [3]:
!pip install idx2numpy

  Preparing metadata (setup.py) ... done
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7904 sha256=784fce923bee31af0eabd2a104ae576f78c25e9210a9503acf763e9b33abf4a8
  Stored in directory: /root/.cache/pip/wheels/e0/f4/e7/643fc5f932ec2ff92997f43f007660feb23f948aa8486f1107
Successfully built idx2numpy


In [3]:
import os

# List all files and directories in /kaggle/working/
print(os.listdir('/kaggle/working/'))

# If you used "Add Data" in Kaggle, list files in the `/kaggle/input/` directory
print(os.listdir('/kaggle/input/'))


['.virtual_documents']
['mnist-dataset']


In [4]:
# Check if the dataset directory exists
dataset_dir = '/kaggle/input/mnist-dataset/'
print(os.listdir(dataset_dir))

['t10k-labels-idx1-ubyte', 'train-images.idx3-ubyte', 't10k-images-idx3-ubyte', 't10k-labels.idx1-ubyte', 't10k-images.idx3-ubyte', 'train-labels.idx1-ubyte', 'train-labels-idx1-ubyte', 'train-images-idx3-ubyte']


In [9]:
import idx2numpy
import os

# Check if the dataset directory exists
dataset_dir = '/kaggle/input/mnist-dataset/'
print(os.listdir(dataset_dir))

# Update paths with the correct filenames
train_images_path = os.path.join(dataset_dir, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
train_labels_path = os.path.join(dataset_dir, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_path = os.path.join(dataset_dir, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_path = os.path.join(dataset_dir, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

# Load data into NumPy arrays
train_images = idx2numpy.convert_from_file(train_images_path)
train_labels = idx2numpy.convert_from_file(train_labels_path)
test_images = idx2numpy.convert_from_file(test_images_path)
test_labels = idx2numpy.convert_from_file(test_labels_path)

# Check the shapes of loaded data
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)


['t10k-labels-idx1-ubyte', 'train-images.idx3-ubyte', 't10k-images-idx3-ubyte', 't10k-labels.idx1-ubyte', 't10k-images.idx3-ubyte', 'train-labels.idx1-ubyte', 'train-labels-idx1-ubyte', 'train-images-idx3-ubyte']
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [26]:

# customize my data for pytorch
class mnist_data(Dataset):
  def __init__(self, images, labels, transform):
    self.images = images
    self.labels = labels
    self.transform = transform
    self.checking_demention = self.transform(images) # just for checking dementiion and transformation to ternosr
  def __len__ (self):
    return len(self.labels)
  def __getitem__(self, index) :
    image = self.images[index]
    label = self.labels[index]
    image = self.transform(image)
    return image, label



In [28]:

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])

In [29]:

train_dataset = mnist_data(train_images, train_labels, transform)
test_dataset = mnist_data(test_images, test_labels, transform)


# create dataloader for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [30]:
print(f"Train Images: {train_dataset.checking_demention.shape}")

Train Images: torch.Size([28, 60000, 28])


In [31]:

# definition of model cnn
class cnn(nn.Module):
  def __init__(self):
    super(cnn, self).__init__()
    # convolution layer and pooling
    self.conv1 = nn.Conv2d(1, 4, 5, stride=1, padding=0)
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(4, 8, 5, stride=1, padding=0)
    self.relu = nn.ReLU()

    # linear layer
    self.l1 = nn.Linear(8*4*4 , 40)
    self.l2 = nn.Linear(40, 20)
    self.outut = nn.Linear(20, 10)

  def forward(self, x):
    x = self.pool1(self.relu(self.conv1(x)))
    x = self.pool1(self.relu(self.conv2(x)))

    x = x.view(-1, 8 * 4 * 4)  # Flatten the tensor
    x = self.relu(self.l1(x))

    x = self.relu(self.l2(x))
    x = self.outut(x)
    return x

model = cnn().to(device)
     

In [32]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # L2 regularization

In [33]:

def train_phase(model, data, loss, optimizer, epochs=10):
  model.train()
  for epoch in range(epochs):
    total_loss = 0.0
    for images, labels in tqdm(data, desc=f"the {epoch+1}/{epochs}"):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      err = loss(outputs, labels)

      optimizer.zero_grad()
      err.backward()
      optimizer.step()

      total_loss += err.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")
      #print(images.shape)
      #print("=======")


In [38]:
from tqdm import tqdm

In [39]:
train_phase(model, train_loader, criterion, optimizer)

the 1/10: 100%|██████████| 938/938 [00:12<00:00, 73.11it/s]


Epoch [1/10], Loss: 0.3812


the 2/10: 100%|██████████| 938/938 [00:12<00:00, 73.55it/s]


Epoch [2/10], Loss: 0.1158


the 3/10: 100%|██████████| 938/938 [00:12<00:00, 73.81it/s]


...

Epoch [8/10], Loss: 0.0481


the 9/10: 100%|██████████| 938/938 [00:12<00:00, 74.06it/s]


Epoch [9/10], Loss: 0.0438


the 10/10: 100%|██████████| 938/938 [00:13<00:00, 71.76it/s]

Epoch [10/10], Loss: 0.0411


In [41]:
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
    return accuracy, f1


In [42]:
accuracy, f1 = evaluate_model(model, test_loader)
     

100%|██████████| 157/157 [00:01<00:00, 110.99it/s]

Accuracy: 0.9850, F1 Score: 0.9850
